In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pandas as pd
import glob
import os
import cv2
import random as rand
import os
import xml.etree.ElementTree as et
import re
import pandas as pd

In [2]:
bases_prontas_path = os.path.join("D:\\","FIA","TCC","BASES","")
print(bases_prontas_path)

D:\FIA\TCC\BASES\


## Criar modelos com transfer learning

In [3]:
baseModel_MobileNetV2 = MobileNetV2(weights="imagenet", include_top=False,
                        input_tensor=Input(shape=(64, 64, 3)))

C:\Users\wesle\anaconda3\envs\py3_6_tensorflow2_1\lib\site-packages\keras_applications\mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [4]:
# construct the head of the model that will be placed on top of the
# the base model
headModel_MobileNetV2 = baseModel_MobileNetV2.output
# headModel_MobileNetV2 = AveragePooling2D(pool_size=(7, 7))(headModel_MobileNetV2)
headModel_MobileNetV2 = AveragePooling2D(pool_size=(2, 2))(headModel_MobileNetV2)
headModel_MobileNetV2 = Flatten(name="flatten")(headModel_MobileNetV2)
headModel_MobileNetV2 = Dense(12288, activation="relu")(headModel_MobileNetV2)
headModel_MobileNetV2 = Dropout(0.5)(headModel_MobileNetV2)
headModel_MobileNetV2 = Dense(4096, activation="relu")(headModel_MobileNetV2)
headModel_MobileNetV2 = Dropout(0.5)(headModel_MobileNetV2)
headModel_MobileNetV2 = Dense(64, activation="relu")(headModel_MobileNetV2)
headModel_MobileNetV2 = Dropout(0.5)(headModel_MobileNetV2)
headModel_MobileNetV2 = Dense(3, activation="softmax")(headModel_MobileNetV2)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model_based_MobileNetV2 = Model(inputs=baseModel_MobileNetV2.input, outputs=headModel_MobileNetV2)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel_MobileNetV2.layers:
    layer.trainable = False

In [5]:
baseModel_MobileNetV2_224 = MobileNetV2(weights="imagenet", include_top=False,
                        input_tensor=Input(shape=(224, 224, 3)))
# construct the head of the model that will be placed on top of the
# the base model
headModel_MobileNetV2_224 = baseModel_MobileNetV2_224.output
headModel_MobileNetV2_224 = AveragePooling2D(pool_size=(7, 7))(headModel_MobileNetV2_224)
headModel_MobileNetV2_224 = Flatten(name="flatten")(headModel_MobileNetV2_224)
headModel_MobileNetV2_224 = Dense(12288, activation="relu")(headModel_MobileNetV2_224)
headModel_MobileNetV2_224 = Dropout(0.5)(headModel_MobileNetV2_224)
headModel_MobileNetV2_224 = Dense(4096, activation="relu")(headModel_MobileNetV2_224)
headModel_MobileNetV2_224 = Dropout(0.5)(headModel_MobileNetV2_224)
headModel_MobileNetV2_224 = Dense(224, activation="relu")(headModel_MobileNetV2_224)
headModel_MobileNetV2_224 = Dropout(0.5)(headModel_MobileNetV2_224)
headModel_MobileNetV2_224 = Dense(3, activation="softmax")(headModel_MobileNetV2_224)


# place the head FC model on top of the base model (this will become
# the actual model we will train)
model_based_MobileNetV2_224 = Model(inputs=baseModel_MobileNetV2_224.input, outputs=headModel_MobileNetV2_224)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel_MobileNetV2_224.layers:
    layer.trainable = False

In [6]:
INIT_LR = 1e-10
EPOCHS = 200
BS = 1
test_size_val = 0.45


aug = ImageDataGenerator(
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    fill_mode="nearest"
    )

In [7]:
print("[INFO] compiling model...")
opt1 = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
# opt2 = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)

[INFO] compiling model...


## Compilar e treinar os modelos criados

In [8]:
dataset = np.load(bases_prontas_path+'mask_dataset_mobilenet_v2_preprocess_input_64_64_3.npy')
# dataset = np.load(bases_prontas_path+'mask_dataset_mobilenet_v2_preprocess_input_224_224_3_2_categorias.npy')
target = np.load(bases_prontas_path+'mask_dataset_labels.npy')
(trainX, testX, trainY, testY) = train_test_split(dataset, target,
                                                  test_size=test_size_val, stratify=target, random_state=42)

In [ ]:
model_based_MobileNetV2.compile(loss="categorical_crossentropy", optimizer="sgd",
              metrics=["accuracy"])

# train the head of the network
print("[INFO] training head...")
H_MobileNetV2 = model_based_MobileNetV2.fit(
# rodar sem o aug para ter um baseline e depois avaliar se vale a pena ter o aug
    aug.flow(trainX, trainY, batch_size=BS),
#     trainX, trainY,
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS)

# executed in 3h 22m 7s, finished 01:31:26 2020-09-04  50 epocas 224, 224,3
# executed in 5h 28m 54s, finished 20:06:13 2020-09-08 100 epocas 64,64,3

"""
opt1 = Adam
Epoch 100/100
15220/15220 [==============================] 
- 414s 27ms/sample - loss: 0.0435 
- accuracy: 0.9865 
- val_loss: 17.6436 
- val_accuracy: 0.3173os 
- ETA: 54s 
- loss: 0 
- ETA: 53s 
- loss: 0.0453 
- ac - ETA: 52s 
- loss: 0.0452
"""

"""
Epoch 100/100 - INIT_LR = 1e-8 - 8h 23m
- loss: 0.2827 - accuracy: 0.8880 - val_loss: 5.9187 - val_accuracy: 0.2993
Epoch 10/10 - INIT_LR = 1e-2
loss: 1.0862 - accuracy: 0.3974 - val_loss: 1.1371 - val_accuracy: 0.3003

"""

[INFO] training head...
  ...
    to  
  ['...']
Train for 11959 steps, validate on 9785 samples
Epoch 1/200
11959/11959 [==============================] - 322s 27ms/step - loss: 1.0957 - accuracy: 0.3980 - val_loss: 1.0901 - val_accuracy: 0.3999
Epoch 2/200
11959/11959 [==============================] - 301s 25ms/step - loss: 1.0907 - accuracy: 0.3994 - val_loss: 1.0946 - val_accuracy: 0.3999
Epoch 3/200
11959/11959 [==============================] - 300s 25ms/step - loss: 1.0906 - accuracy: 0.4000 - val_loss: 1.0896 - val_accuracy: 0.3999
Epoch 4/200
11959/11959 [==============================] - 299s 25ms/step - loss: 1.0911 - accuracy: 0.3994 - val_loss: 1.0890 - val_accuracy: 0.3999
Epoch 5/200
11959/11959 [==============================] - 298s 25ms/step - loss: 1.0905 - accuracy: 0.3997 - val_loss: 1.0917 - val_accuracy: 0.3999
Epoch 6/200
11959/11959 [==============================] - 297s 25ms/step - loss: 1.0909 - accuracy: 0.3995 - val_loss: 1.0892 - val_accuracy: 0.3999
Epo

11959/11959 [==============================] - 298s 25ms/step - loss: 1.0907 - accuracy: 0.3989 - val_loss: 1.0896 - val_accuracy: 0.3999
Epoch 55/200
11959/11959 [==============================] - 298s 25ms/step - loss: 1.0910 - accuracy: 0.3998 - val_loss: 1.0908 - val_accuracy: 0.3999
Epoch 56/200
11959/11959 [==============================] - 297s 25ms/step - loss: 1.0909 - accuracy: 0.3998 - val_loss: 1.0899 - val_accuracy: 0.3999
Epoch 57/200
11959/11959 [==============================] - 297s 25ms/step - loss: 1.0905 - accuracy: 0.3998 - val_loss: 1.0890 - val_accuracy: 0.3999
Epoch 58/200
11959/11959 [==============================] - 297s 25ms/step - loss: 1.0906 - accuracy: 0.3996 - val_loss: 1.0928 - val_accuracy: 0.3999
Epoch 59/200
11959/11959 [==============================] - 297s 25ms/step - loss: 1.0903 - accuracy: 0.3996 - val_loss: 1.0907 - val_accuracy: 0.3999
Epoch 60/200
11959/11959 [==============================] - 297s 25ms/step - loss: 1.0909 - accuracy: 0.399

In [ ]:
# dataset = np.load(bases_prontas_path+'mask_dataset_mobilenet_v2_preprocess_input_64_64_3.npy')
dataset = np.load(bases_prontas_path+'mask_dataset_mobilenet_v2_preprocess_input_224_224_3.npy')
target = np.load(bases_prontas_path+'mask_dataset_labels.npy')
(trainX, testX, trainY, testY) = train_test_split(dataset, target,
                                                  test_size=test_size_val, stratify=target, random_state=42)

model_based_MobileNetV2_224.compile(loss="categorical_crossentropy", optimizer=opt1,
              metrics=["accuracy"])

# train the head of the network
print("[INFO] training head...")
H_MobileNetV2_224 = model_based_MobileNetV2_224.fit(
# rodar sem o aug para ter um baseline e depois avaliar se vale a pena ter o aug
    aug.flow(trainX, trainY, batch_size=BS),
#     trainX, trainY,
    steps_per_epoch=len(trainX2) // BS,
    validation_data=(testX2, testY2),
    validation_steps=len(testX2) // BS,
    epochs=EPOCHS)

## Avaliar os modelos

In [ ]:
print("[INFO] evaluating network...")
lb = ["without_mask","mask_weared_incorrect","with_mask"]
def Avaliando_modelo(model, NWHead = None,x_test=testX, y_test=testY):
    predIdxs = model.predict(x_test, batch_size=32)

    # for each image in the testing set we need to find the index of the
    # label with corresponding largest predicted probability
    predIdxs = np.argmax(predIdxs, axis=1)

    # show a nicely formatted classification report
    print(classification_report(y_test.argmax(axis=1), predIdxs,
                                target_names=lb))

    # serialize the model to disk
    print("[INFO] saving mask detector model...")

    if NWHead:
        # plot the training loss and accuracy
        N = EPOCHS
        plt.style.use("ggplot")
        plt.figure()
        plt.plot(np.arange(0, N), NWHead.history["loss"], label="train_loss")
        plt.plot(np.arange(0, N), NWHead.history["val_loss"], label="val_loss")
        plt.plot(np.arange(0, N), NWHead.history["accuracy"], label="train_acc")
        plt.plot(np.arange(0, N), NWHead.history["val_accuracy"], label="val_acc")
        plt.title("Training Loss and Accuracy")
        plt.xlabel("Epoch #")
        plt.ylabel("Loss/Accuracy")
        plt.legend(loc="lower left")
        plt.show()
    return classification_report(y_test.argmax(axis=1), predIdxs,
                                target_names=lb,output_dict=True)

In [ ]:
Avaliando_modelo(model_based_MobileNetV2, H_MobileNetV2, testX, testY)

In [ ]:
Avaliando_modelo(model_based_MobileNetV2_224, H_MobileNetV2_224, testX,  testY)

## salvar os pesos dos modelos para não precisar salvar de novo

In [ ]:
model_path = os.path.join("D:\\","FIA","TCC","Meus_Modelos","")
print(model_path)

In [ ]:
# https://www.tensorflow.org/tutorials/keras/save_and_load
# Save the weights
model_based_MobileNetV2.save_weights(model_path+'model_based_MobileNetV2_64_checkpoint')
model_based_MobileNetV2_224.save_weights(model_path+'model_based_MobileNetV2_224_checkpoint')
